In [2]:
! pip install spacy
! python -m spacy download en_core_web_sm

  Using cached spacy-3.8.11-cp311-cp311-macosx_11_0_arm64.whl.metadata (27 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.15-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.3 kB)
  Using cached cymem-2.0.13-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.7 kB)
  Using cached preshed-3.0.12-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.5 kB)
  Using cached thinc-8.3.10-cp311-cp311-macosx_11_0_arm64.whl.metadata (15 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.5.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer_slim-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached blis-1.3.3-cp311-cp311-macosx_11_0_arm64

In [ ]:
import random
import spacy

nlp = spacy.load("en_core_web_sm")

class MemoryAgent:
    # We use these to track when the user refers back to nouns that were said before.
    PRONOUNS = {"it", "they", "he", "she", "this", "that", "these", "those", "i", "me", "my", "we", "us", "our", "you", "your"}

    def __init__(self, max_memory_length):
        self.memory = []
        self.max_memory_length = max_memory_length

    def user_update(self, content: str):
        features = self._extract_features(content)
        if self.memory and any(s.lower() in self.PRONOUNS for s in features["subj"]):
            prev = self.memory[-1]
            features["resolved_refs"] = prev.get("subj", []) + prev.get("obj", []) + prev.get("entities", [])
        self.memory.append(features)
        if len(self.memory) > self.max_memory_length:
            self.memory = self.memory[-self.max_memory_length:]

    def get_full_memory_summary(self, n_contrast: int = 1) -> str:
        if not self.memory:
            return ""

        current_idx = len(self.memory) - 1
        relevant_idx = self._get_relevant_indices(current_idx)
        non_relevant_idx = [i for i in range(current_idx + 1) if i not in relevant_idx]

        # Group non-relevant by topic and randomly select n
        contrast_groups = []
        used = set()
        for idx in sorted(non_relevant_idx, reverse=True):
            if idx in used:
                continue
            group = [i for i in self._get_relevant_indices(idx) if i in non_relevant_idx]
            if group:
                contrast_groups.append(group)
                used.update(group)

        selected_contrasts = random.sample(contrast_groups, min(n_contrast, len(contrast_groups)))

        parts = [self._summarize([self.memory[i] for i in relevant_idx])]
        for group in selected_contrasts:
            parts.append(self._summarize([self.memory[i] for i in group]))

        return " | ".join(filter(None, parts))

    def _get_relevant_indices(self, target_idx: int) -> set[int]:
        if target_idx < 0 or target_idx >= len(self.memory):
            return set()
        target_refs = self._get_refs(self.memory[target_idx])
        return {i for i in range(target_idx + 1) if not target_refs.isdisjoint(self._get_refs(self.memory[i]))}

    def _get_refs(self, f: dict) -> set:
        all_refs = f.get("subj", []) + f.get("obj", []) + f.get("entities", []) + f.get("resolved_refs", [])
        return {r for r in all_refs if r.lower() not in self.PRONOUNS}

    def _summarize(self, memory_list: list[dict]) -> str:
        if not memory_list:
            return ""
        tuples = []
        for f in memory_list:
            tuples.append((
                " ".join(w for w in f.get("subj", []) if w.lower() not in self.PRONOUNS),
                " ".join(f.get("verbs", [])),
                " ".join(f.get("adjectives", [])),
                " ".join(w for w in f.get("obj", []) if w.lower() not in self.PRONOUNS),
                " ".join(f.get("entities", [])),
            ))
        # We basically create a string with subjects, verbs, adjectives, ... with this zip.
        return " ".join(filter(None, (" ".join(filter(None, t)) for t in zip(*tuples))))

    def _extract_features(self, sentence: str) -> dict:
        doc = nlp(sentence)
        return {
            "entities": [e.text for e in doc.ents],
            "subj": [t.text for t in doc if "subj" in t.dep_],
            "obj": [t.text for t in doc if "obj" in t.dep_],
            "adjectives": [t.text for t in doc if t.pos_ == "ADJ"],
            "verbs": [t.lemma_ for t in doc if t.pos_ == "VERB"],
            "past_tense": any(t.tag_ == "VBD" for t in doc),
            "negated": any(t.dep_ == "neg" for t in doc),
            "numbers": [t.text for t in doc if t.like_num],
        }
    
memory = MemoryAgent(10)

memory.user_update("I live in a beautiful house currently.")
print(memory.get_full_memory_summary(3))
memory.user_update("It was pretty cheap")
print(memory.get_full_memory_summary(3))

memory.user_update("My phone is also new")
print(memory.get_full_memory_summary(3))

memory.user_update("It has a great camera")
print(memory.get_full_memory_summary(3))

memory.user_update("I love my dog")
print(memory.get_full_memory_summary(3))

memory.user_update("He is very playful")
print(memory.get_full_memory_summary(3))

memory.user_update("Back to the phone, it also has good battery")
print(memory.get_full_memory_summary(3))

live beautiful house
live beautiful cheap house
phone new | live beautiful cheap house
phone have new great camera | live beautiful cheap house
have love great camera dog | live beautiful cheap house | phone new
love playful dog | phone have new great camera | live beautiful cheap house
phone have have new great good camera phone battery | love playful dog | live beautiful cheap house
